In [1]:
from preprocessing import df, pd

In [2]:
df

,No.,Time,Source IP,Destination IP,Protocol,Length,Source Port,Destination Port,Resolved Source Mac,Source Mac Address,TCP Flags
0,1,0.000000,192.168.0.107,192.168.0.255,UDP,305,59199.0,54915.0,IntelCor_e0:2d:d3,a0:88:69:e0:2d:d3,None
1,2,0.102253,192.168.0.107,192.168.0.255,UDP,305,54915.0,54915.0,IntelCor_e0:2d:d3,a0:88:69:e0:2d:d3,None
2,3,0.244179,192.168.0.100,192.168.0.2,DNS,81,61098.0,53.0,Apple_56:69:c0,8c:85:90:56:69:c0,None
3,4,0.305297,IntelCor_6c:2f:50,Broadcast,ARP,42,NaN,NaN,IntelCor_6c:2f:50,cc:3d:82:6c:2f:50,None
4,5,0.308302,192.168.0.2,192.168.0.100,ICMP,109,61098.0,53.0,D-LinkIn_12:86:d8,3c:1e:04:12:86:d8,None
...,...,...,...,...,...,...,...,...,...,...,...
117328,117329,71.852720,192.168.0.100,192.168.0.121,TCP,66,80.0,64484.0,Apple_56:69:c0,8c:85:90:56:69:c0,"(SYN, ACK)"
117329,117330,71.852737,192.168.0.100,192.168.0.121,TCP,66,80.0,64463.0,Apple_56:69:c0,8c:85:90:56:69:c0,"(SYN, ACK)"
117330,117331,71.852776,192.168.0.100,192.168.0.121,TCP,66,80.0,64464.0,Apple_56:69:c0,8c:85:90:56:69:c0,"(SYN, ACK)"
117331,117332,71.866342,192.168.0.100,192.168.0.104,TCP,460,80.0,58082.0,Apple_56:69:c0,8c:85:90:56:69:c0,"(FIN, PSH, ACK)"
